In [69]:
import pandas as pd
import numpy as np
import time
import requests
import urllib.request
from urllib.parse import quote_plus
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import unicodedata
import time

In [3]:
!pip3 install selenium

     |████████████████████████████████| 904 kB 2.7 MB/s eta 0:00:01


In [17]:
!pip3 install xlrd==1.2.0

In [58]:
dining_list = pd.read_csv('../data/yonsei_dining_list.csv')
dining_list

,dining_id,total_name,dining_name,branch_name,address,latitude,longitude,distance
0,0,이때,이때,NaN,서울특별시 마포구 동교로38안길 7,37.562176,126.924660,1.294640
1,1,대원식당,대원식당,NaN,서울특별시 서대문구 이화여대길 64,37.559250,126.944871,0.920894
2,2,SRS버거킹 이대점,SRS버거킹,이대점,서울특별시 서대문구 이화여대길 30,37.558241,126.945915,1.065890
3,3,벌떼식당,벌떼식당,NaN,서울특별시 마포구 신촌로20길 12,37.554604,126.938169,1.253088
4,4,ZINO본점,ZINO본점,본점,서울특별시 서대문구 연대동문길 45,37.564409,126.944391,0.536427
...,...,...,...,...,...,...,...,...
1923,1923,앵란이모네포차,앵란이모네포차,NaN,서울특별시 서대문구 연세로5가길 16,37.557394,126.935832,0.973146
1924,1924,BARREGGIOSEOUL,BARREGGIOSEOUL,NaN,서울특별시 서대문구 연세로5다길 9,37.556295,126.934908,1.112823
1925,1925,하이 라오상,하이,라오상,서울특별시 서대문구 연세로12길 27,37.559094,126.938546,0.753218
1926,1926,바른치킨 북아현점,바른치킨,북아현점,서울특별시 서대문구 이화여대8길 123,37.563191,126.950283,1.072739


In [59]:
def add_region(x) :
    if '신촌' in x :
        return x
    else :
        return '신촌'+ x
        

In [60]:
dining_list['new_dining_name'] = dining_list['dining_name'].map(lambda x: add_region(x))
dining_list = dining_list[['dining_id','new_dining_name','total_name','dining_name','branch_name','address']]
dining_list

,dining_id,new_dining_name,total_name,dining_name,branch_name,address
0,0,신촌이때,이때,이때,NaN,서울특별시 마포구 동교로38안길 7
1,1,신촌대원식당,대원식당,대원식당,NaN,서울특별시 서대문구 이화여대길 64
2,2,신촌SRS버거킹,SRS버거킹 이대점,SRS버거킹,이대점,서울특별시 서대문구 이화여대길 30
3,3,신촌벌떼식당,벌떼식당,벌떼식당,NaN,서울특별시 마포구 신촌로20길 12
4,4,신촌ZINO본점,ZINO본점,ZINO본점,본점,서울특별시 서대문구 연대동문길 45
...,...,...,...,...,...,...
1923,1923,신촌앵란이모네포차,앵란이모네포차,앵란이모네포차,NaN,서울특별시 서대문구 연세로5가길 16
1924,1924,신촌BARREGGIOSEOUL,BARREGGIOSEOUL,BARREGGIOSEOUL,NaN,서울특별시 서대문구 연세로5다길 9
1925,1925,신촌하이,하이 라오상,하이,라오상,서울특별시 서대문구 연세로12길 27
1926,1926,신촌바른치킨,바른치킨 북아현점,바른치킨,북아현점,서울특별시 서대문구 이화여대8길 123


In [67]:
### 설정값
#이미지를 따로 저장할 'img'라는 디렉토리를 미리 생성해주세요.

path = '/Users/pdh/TasteMaps/TasteMap/crawler/chromedriver' #크롬드라이버 위치

ID = '인스타 아이디' #인스타 id
PW = '인스타 비밀번호' #인스타 비밀번호

SCROLL_NUM = 13 #얼마나 스크롤할 지 정하기(스크롤을 내려 게시물을 많이 확보해야 합니다. 값이 최소 1이어야 합니다.)
CRAWLING_NUM = 100 #크롤링할 게시물 개수

tag_list = dining_list[:50] #검색할 range 입력
tag_list

,dining_id,new_dining_name,total_name,dining_name,branch_name,address
0,0,신촌이때,이때,이때,NaN,서울특별시 마포구 동교로38안길 7
1,1,신촌대원식당,대원식당,대원식당,NaN,서울특별시 서대문구 이화여대길 64
2,2,신촌SRS버거킹,SRS버거킹 이대점,SRS버거킹,이대점,서울특별시 서대문구 이화여대길 30
3,3,신촌벌떼식당,벌떼식당,벌떼식당,NaN,서울특별시 마포구 신촌로20길 12
4,4,신촌ZINO본점,ZINO본점,ZINO본점,본점,서울특별시 서대문구 연대동문길 45
5,5,신촌진국수,진국수,진국수,NaN,서울특별시 서대문구 이화여대7길 29
6,6,신촌고구려,고구려,고구려,NaN,서울특별시 서대문구 명물길 23
7,7,이랜드파크외식신촌그랜드마트,이랜드파크외식신촌그랜드마트,이랜드파크외식신촌그랜드마트,NaN,서울특별시 마포구 신촌로 94
8,8,신촌오므토토마토,오므토토마토 이대점,오므토토마토,이대점,서울특별시 서대문구 신촌역로 22-8
9,9,신촌파주식당,파주식당,파주식당,NaN,서울특별시 서대문구 이화여대7길 42


In [71]:
def get_url(driver, tag):
    url = "https://www.instagram.com/explore/tags/"
    url = url + quote_plus(tag['new_dining_name'])
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = bs(html, 'lxml')

In [64]:
def scroll(drive, scroll_num):
    #컨텐츠가 없으면 False 반환 -> 다음 태그로 넘어감
    try :  
        driver.find_element_by_class_name('_9AhH0')
    except :
        print('#################No Content##################')
        return False
    #SCROLL_NUM만큼 맨 밑으로 스크롤하고 맨 위로 돌아오기
    for _ in range(scroll_num):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
        
    driver.execute_script('window.scrollTo(0,0);')
    #첫 번째 게시물
    
    driver.find_element_by_class_name('_9AhH0').click()
    return True

In [65]:
def crawl(driver, crawl_num, tag):
    
    n = 0
    
    for i in range(crawl_num):
        html = driver.page_source
        soup = bs(html, 'lxml')
        time.sleep(1)

        contents = driver.find_elements_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
        if len(contents) == 0:
            contents = np.nan
        else:
            contents = contents[0].text
            contents = unicodedata.normalize('NFC', contents)

        dining_ids.append(tag['dining_id'])
        total_names.append(tag['total_name'])
        sources.append('인스타그램')
        contents_list.append(contents)

        #다음 게시물 , 없으면 종료
        try : 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(1)
        except :
            return
        
        n += 1

In [72]:
### 실행코드

dining_ids = []
total_names = []
sources = []
rate = []
contents_list = []
start = time.time()

driver = webdriver.Chrome(path)
driver.maximize_window()
driver.get('https://instagram.com/explore')
time.sleep(1)

driver.find_element_by_name('username').send_keys(ID)
driver.find_element_by_name('password').send_keys(PW)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(3)

for index,tag in tag_list.iterrows():
    print(tag['new_dining_name'])
    get_url(driver, tag)
    time.sleep(1)
    
    if not scroll(driver, SCROLL_NUM) :
        continue
    crawl(driver, CRAWLING_NUM, tag)
    
    
print("총 걸린 시간 :", time.time() - start)

### 저장
data = {'dining_id':dining_ids,
        'total_name':total_names,
         'review':contents_list,
         'source': sources}
    
df = pd.DataFrame(data)
df.to_csv("test_data.csv", index=True, encoding='utf-8-sig')

신촌이때
#################No Content##################
신촌대원식당
#################No Content##################
신촌SRS버거킹
#################No Content##################
신촌벌떼식당
신촌ZINO본점
#################No Content##################
신촌진국수
#################No Content##################
신촌고구려
#################No Content##################
이랜드파크외식신촌그랜드마트
#################No Content##################
신촌오므토토마토
#################No Content##################
신촌파주식당
#################No Content##################
신촌리화인와플
#################No Content##################
해마로파파이스신촌중앙
#################No Content##################
신촌예강설렁탕
#################No Content##################
신촌춘천집닭갈비막국수
신촌최고네
#################No Content##################
신촌케이즈
#################No Content##################
신촌그라땅12
#################No Content##################
신촌코피티암
#################No Content##################
신촌기꾸초밥
신촌켈리인더볼
#################No Content##################
신촌형제갈비
신촌도스타코스
신촌향꽃민속주점
#################No Content#####

In [76]:
### 저장
data = {'dining_id':dining_ids,
        'total_name':total_names,
         'review':contents_list,
         'source': sources}
    
df = pd.DataFrame(data)
df.to_csv("test_data.csv", index=True, encoding='utf-8-sig')

In [77]:
df[df['dining_id'] == 20]

,dining_id,total_name,review,source
7,20,신촌형제갈비,기가막힌다 진짜\n.\n.\n.\n#형제갈비 #신촌형제갈비 #소갈비 #소갈비살 #양...,인스타그램
8,20,신촌형제갈비,"#신촌형제갈비\n\n불고기,갈비 맛집!!\n맛도 맛이지만 한국느낌의\n분위기가 너무...",인스타그램
9,20,신촌형제갈비,일요일 폭풍먹방 후 오늘 아침은 방탄커피\n영화보고 2시에 첫끼. 뜯는 맛이 있는 ...,인스타그램
10,20,신촌형제갈비,#신촌형제갈비\n\n일년에 고기 땡기는 날이 몇 번 안되는데 오늘이 그날! 오늘따라...,인스타그램
11,20,신촌형제갈비,"+ 2020.03.15 🇰🇷 [서울] 형제갈비, 테일러커피 서교 3호점, 클로리스,...",인스타그램
...,...,...,...,...
102,20,신촌형제갈비,#koreankalbi #koreankabirestaurant #미국사는우리가한국에...,인스타그램
103,20,신촌형제갈비,난 사진을 못찍는다\n#신촌형제갈비,인스타그램
104,20,신촌형제갈비,#평양냉면34 #형제갈비 #신촌형제갈비 #10000원,인스타그램
105,20,신촌형제갈비,스승의 날에 교수님과😎\n매번 짝꿍언제 데리고 올꺼냐고 물어보시는데...\n그건 저...,인스타그램
